In [1]:
from components.fuzzymatching import FuzzyMatchingFactory
from core.config import Environment, Config


matching_tool_config = {'csv_path': './backend/db/keyframes.csv', 
                        'env_dir': Environment()}
ocr_matcher = FuzzyMatchingFactory.produce('rapidwuzzy', **matching_tool_config)


/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [2]:
from components.llms import AgentFactory
from components.llms.prompts import EXTRACT_KEYWORDS
from schema.llm_response import Keyword

import os 
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm_config = {'api_key': GOOGLE_API_KEY, 
              'response_setting': {
                  "response_mime_type":'application/json', 
                  "response_schema": list[Keyword]
              }}
llm_agent = AgentFactory.produce(provider='gemini', 
                                 **llm_config)
llm_agent

In [3]:
query = "Đoạn giới thiệu về lễ hội Việt - Nhật. Trong lễ hội có các phân cảnh những chiếc lồng đèn kiểu Nhật cùng những chiếc dù Wagasa. Phân đoạn giữa có hình ảnh của chú mèo máy Doraemon khổng lồ và có những người xung quanh chụp ảnh với nó. Phân đoạn sau đó có hình ảnh một ban nhạc Nhật chơi các nhạc cụ truyền thống trên sân khấu. Xuyên suốt đoạn giới thiệu là một số đoạn phỏng vấn những người tham gia lễ hội. Hỏi có tổng cộng bao nhiêu người khác nhau được phỏng vấn trong đoạn giới thiệu trên?"

In [4]:
import json

kw_v3 = llm_agent.run(EXTRACT_KEYWORDS + '\n' + query)
kw_v3 = json.loads(kw_v3)
kw_v3 = [d['keyword'] for d in kw_v3]
kw_v3

['Việt - Nhật Festival',
 'Japanese lanterns',
 'Wagasa umbrellas',
 'Doraemon',
 'Japanese music',
 'traditional instruments',
 'festival attendees']

In [5]:
matching_paths_v3 = []

for kw in kw_v3: 
    result = ocr_matcher.run(kw)
    img_paths = ocr_matcher.get_image_paths(result)
    matching_paths_v3 += img_paths

print(len(matching_paths_v3))
matching_paths_v3 = list(set(matching_paths_v3))
print(len(matching_paths_v3))
print(matching_paths_v3)

70
57
['L24_V032-16779.webp', 'L01_V023-16435.webp', 'L24_V009-11008.webp', 'L24_V002-03835.webp', 'L22_V016-32655.webp', 'L09_V014-31190.webp', 'L24_V043-10716.webp', 'L10_V024-00419.webp', 'L09_V010-00657.webp', 'L20_V016-00007.webp', 'L01_V031-21436.webp', 'L24_V024-02104.webp', 'L21_V027-12655.webp', 'L20_V013-12582.webp', 'L24_V030-17737.webp', 'L24_V031-07816.webp', 'L24_V004-19383.webp', 'L24_V002-05810.webp', 'L18_V008-00318.webp', 'L10_V008-29727.webp', 'L24_V028-12507.webp', 'L24_V013-09371.webp', 'L24_V031-07865.webp', 'L24_V029-15532.webp', 'L24_V037-15045.webp', 'L20_V019-30498.webp', 'L24_V006-08591.webp', 'L24_V014-03893.webp', 'L02_V030-15045.webp', 'L07_V013-00529.webp', 'L23_V021-11160.webp', 'L13_V022-20516.webp', 'L24_V011-02194.webp', 'L24_V032-10966.webp', 'L24_V009-05306.webp', 'L24_V004-14444.webp', 'L19_V024-31339.webp', 'L24_V014-08476.webp', 'L24_V008-02415.webp', 'L20_V019-24400.webp', 'L05_V024-18145.webp', 'L13_V024-01560.webp', 'L20_V017-33624.webp', 'L24

In [6]:
import json
env_dir = Environment()
with open(os.path.join(env_dir.root, env_dir.db_root, env_dir.vid_url), 'r') as f:
    vid_url = json.load(f)

with open(os.path.join(env_dir.root, env_dir.db_root, env_dir.url_fps), 'r') as f:
    url_fps = json.load(f)

In [14]:
def search_by_ocr(query:str, top_k, matching_tool, llm, vid_url:dict, url_fps:dict): 
    """
    Searching by keywords using OCR.
    """

    # Pick up keywords from the original query
    extract_kw_query = EXTRACT_KEYWORDS + "\n" + query
    retries = 3
    try: 
        
        keywords = llm.run(extract_kw_query)
        # print(keywords)
        # print(type(keywords))
        if isinstance(keywords, str): 
            while retries >= 0: 
                # Normally, it executes 2 times and break.
                keywords = json.loads(keywords)
                # print(type(keywords))
                if type(keywords) == dict or type(keywords) == list: 
                    break
                retries -= 1
        keywords = [d['keyword'] for d in keywords]
        # print(keywords)

    except Exception as e: 
        raise json.JSONDecodeError(e)

In [7]:
vid_urls = []
# embed_urls = []
frames = []
# image_paths = [image_path.split('/')[-1] for image_path in matching_paths]
for img_id in matching_paths_v3: 
    vid_id = img_id.split('.')[0]
    vid_name, frame = vid_id.split('-')
    url = vid_url[vid_name] + '&t=' + str(int(int(frame)/url_fps[vid_url[vid_name]]))
    # embed_url = vid_url[vid_name].replace('watch?v=', 'embed/') # TODO: make this execuate from frontend
    vid_urls.append(url)
    # embed_urls.append(embed_url)
    frames.append(frame)


In [15]:

results = search_by_ocr(query=query, 
                        top_k=10, 
                        matching_tool=ocr_matcher, 
                        llm=llm_agent, 
                        vid_url=vid_url, 
                        url_fps=url_fps)
results

In [31]:
from components.llms import AgentFactory
from schema.llm_response import RefineQuery

import os 
from dotenv import load_dotenv

load_dotenv()


llm_config = {'api_key': os.getenv("GOOGLE_API_KEY"), 
              'response_setting': {
                  "response_mime_type":'application/json', 
                  "response_schema": RefineQuery
              }}
refine_agent = AgentFactory.produce(provider='gemini', 
                                 **llm_config)
refine_agent

In [35]:
QUERY_REWRITE = """Provide a concise and specific search query for an event retrieval engine to find relevant events based on the following prompt. Remember to keep the language and intention of the original prompt.

Prompt: {prompt}

Refine Query: 

"""

HYPOTHETICAL_REWRITE = """Act as a prompt engineer skilled at refining prompts for an event retrieval engine. Your task is to refine the following prompt to optimize results.  Follow these steps:

1. **Keyword Extraction:** Identify the semantically relevant keywords within the input prompt.  Prioritize keywords that directly relate to events, locations, dates, or key figures.

2. **Contextual Analysis:** Analyze the keywords in relation to the overall topic of the prompt.  Infer the implicit meanings and potential relationships between the keywords.  Consider what specific information the retrieval engine might need to find relevant events.

3. **Refined Prompt Generation:**  Rewrite the input prompt based on your keyword analysis and contextual understanding.  The refined prompt should be a concise and accurate query that explicitly targets the desired information.

Remember to keep the language and intention of the original prompt.

Prompt: {prompt}

Refined Query:
"""

In [36]:
result = refine_agent.run(HYPOTHETICAL_REWRITE.format(prompt=query))
result = json.loads(result)
result['refine_response']

'Tìm số lượng người được phỏng vấn trong đoạn giới thiệu lễ hội Việt - Nhật, tập trung vào các phân cảnh có lồng đèn Nhật, dù Wagasa, mèo máy Doraemon khổng lồ, và ban nhạc Nhật chơi nhạc cụ truyền thống.'

In [2]:
import os 
import json

def get_video_metadata(vid_idx, media_dir): 
    vid_name = '.'.join([vid_idx, 'json']) # File is in .json format
    vid_metadata = json.loads(open(os.path.join(media_dir, vid_name), 'r').read())
    return vid_metadata


vid_idx = 'L18_V015'
media_dir = './db/media-info'

get_video_metadata(vid_idx=vid_idx, 
                   media_dir=media_dir)

{'author': '60 Giây Official',
 'channel_id': 'UCRjzfa1E0gA50lvDQipbDMg',
 'channel_url': 'https://www.youtube.com/channel/UCRjzfa1E0gA50lvDQipbDMg',
 'description': '60 Giây Chiều - Ngày 15/06/2024 - HTV Tin Tức Mới Nhất 2024\n► Đăng ký KÊNH để xem Tin Tức Mới Nhất: https://bit.ly/2HoUna4\nKênh Tin Tức Thời Sự 60 Giây Là Kênh Tổng Hợp Tin Tức - Sự Kiện - Giải trí Nhanh Nhất Việt Nam.\n\n©️ Bản quyền Tin Tức thuộc HTV - Đài Truyền hình TP.HCM\n©️ The copyright of this video belongs to HTV - Ho Chi Minh City Television\n#TintucthoisuVietnam\n#HTVTintuc #HTVnews #HTV60s #60Giay #60s\n------------\nXem TV online tại Truyền hình HTVC:\n✅ Web / Wap mobile :\n► https://hplus.com.vn\n► https://htvc.com.vn/\n► https://htvc.tv/\n✅ App mobile/TV HTVC :\n► IOS: https://goo.gl/Dih3DB\n► Android: https://goo.gl/XGBzxg\n► Smart TV /STB : Tải HTVC for Android TV\n.\n✅ Theo dõi HTVC trên Mạng xã hội :\n- Facebook Show : https://www.facebook.com/HPlusOfficial\n- Instagram Show : https://www.instagram.c